In [ ]:
import pandas as pd
import random
import os
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMRegressor

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(37) # Seed 고정

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/LG Aimers 2기/데이터/train.csv', parse_dates=['TIMESTAMP'])
test_df = pd.read_csv('/content/drive/MyDrive/LG Aimers 2기/데이터/test.csv', parse_dates=['TIMESTAMP'])

train_df.index = train_df['TIMESTAMP']
test_df.index = test_df['TIMESTAMP']

train_df = train_df.drop(columns=['TIMESTAMP','PRODUCT_ID'])
test_df = test_df.drop(columns=['TIMESTAMP','PRODUCT_ID'])

In [ ]:
train_x = train_df.drop(columns=['Y_Class', 'Y_Quality'])
train_y = train_df['Y_Class']

* train_x 데이터에 Robust Scaling 적용

In [ ]:
train_x.head()

,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
TIMESTAMP,,,,,,,,,,,,,,,,,,,,,
2022-06-13 05:14:00,T050304,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,39.34,40.89,32.56,34.09,77.77,NaN,NaN,NaN,NaN,NaN
2022-06-13 05:22:00,T050307,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,38.89,42.82,43.92,35.34,72.55,NaN,NaN,NaN,NaN,NaN
2022-06-13 05:30:00,T050304,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,39.19,36.65,42.47,36.53,78.35,NaN,NaN,NaN,NaN,NaN
2022-06-13 05:39:00,T050307,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,37.74,39.17,52.17,30.58,71.78,NaN,NaN,NaN,NaN,NaN
2022-06-13 05:47:00,T050304,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,38.70,41.89,46.93,33.09,76.97,NaN,NaN,NaN,NaN,NaN


In [ ]:
from sklearn.preprocessing import RobustScaler

train_robust = train_x.iloc[:,2:]

robust_scaler = RobustScaler()

fitted = robust_scaler.fit(train_robust)

train_robust = robust_scaler.transform(train_robust)

train_robust = pd.DataFrame(train_robust, columns = train_x.iloc[:,2:].columns, index = train_x.index)

/usr/local/lib/python3.8/dist-packages/numpy/lib/nanfunctions.py:1119: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(a, func=_nanmedian, axis=axis, out=out,
/usr/local/lib/python3.8/dist-packages/numpy/lib/nanfunctions.py:1374: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(


In [ ]:
train_robust.head()

,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
TIMESTAMP,,,,,,,,,,,,,,,,,,,,,
2022-06-13 05:14:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-2.427402,-2.219188,-1.468531,-3.042011,3.639769,NaN,NaN,NaN,NaN,NaN
2022-06-13 05:22:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-2.504955,-1.934317,-0.585859,-2.869835,2.135447,NaN,NaN,NaN,NaN,NaN
2022-06-13 05:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-2.453253,-2.845018,-0.698524,-2.705923,3.806916,NaN,NaN,NaN,NaN,NaN
2022-06-13 05:39:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-2.703145,-2.473063,0.055167,-3.525482,1.913545,NaN,NaN,NaN,NaN,NaN
2022-06-13 05:47:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-2.537699,-2.071587,-0.351981,-3.179752,3.409222,NaN,NaN,NaN,NaN,NaN


In [ ]:
train_x_robust = pd.concat([train_x.iloc[:,0:2], train_robust], axis=1)

train_x_robust.head()

,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
TIMESTAMP,,,,,,,,,,,,,,,,,,,,,
2022-06-13 05:14:00,T050304,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-2.427402,-2.219188,-1.468531,-3.042011,3.639769,NaN,NaN,NaN,NaN,NaN
2022-06-13 05:22:00,T050307,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-2.504955,-1.934317,-0.585859,-2.869835,2.135447,NaN,NaN,NaN,NaN,NaN
2022-06-13 05:30:00,T050304,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-2.453253,-2.845018,-0.698524,-2.705923,3.806916,NaN,NaN,NaN,NaN,NaN
2022-06-13 05:39:00,T050307,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-2.703145,-2.473063,0.055167,-3.525482,1.913545,NaN,NaN,NaN,NaN,NaN
2022-06-13 05:47:00,T050304,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-2.537699,-2.071587,-0.351981,-3.179752,3.409222,NaN,NaN,NaN,NaN,NaN


In [ ]:
# 이후 scaling 진행한 데이터로 catboostclassifier 분류모델 학습

!pip install catboost
from catboost import CatBoostClassifier

cat2 = CatBoostClassifier(iterations=100, random_state=37)

cat2.fit(train_x_robust, train_y, cat_features=np.where(train_x_robust.dtypes != np.float)[0])

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 10.6 MB/s eta 0:00:00


<ipython-input-10-2e7e03cebe52>:8: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  cat2.fit(train_x_robust, train_y, cat_features=np.where(train_x_robust.dtypes != np.float)[0])


Learning rate set to 0.5
0:	learn: 0.8305971	total: 2.16s	remaining: 3m 33s
1:	learn: 0.7578487	total: 2.83s	remaining: 2m 18s
2:	learn: 0.7172675	total: 3.54s	remaining: 1m 54s
3:	learn: 0.6909057	total: 4.18s	remaining: 1m 40s
4:	learn: 0.6743387	total: 4.83s	remaining: 1m 31s
5:	learn: 0.6561558	total: 5.48s	remaining: 1m 25s
6:	learn: 0.6344991	total: 6.1s	remaining: 1m 21s
7:	learn: 0.6130517	total: 6.69s	remaining: 1m 16s
8:	learn: 0.5944930	total: 7.09s	remaining: 1m 11s
9:	learn: 0.5864673	total: 7.46s	remaining: 1m 7s
10:	learn: 0.5774144	total: 7.83s	remaining: 1m 3s
11:	learn: 0.5643013	total: 8.2s	remaining: 1m
12:	learn: 0.5459619	total: 8.58s	remaining: 57.4s
13:	learn: 0.5385314	total: 8.94s	remaining: 54.9s
14:	learn: 0.5201931	total: 9.32s	remaining: 52.8s
15:	learn: 0.5062757	total: 9.69s	remaining: 50.9s
16:	learn: 0.4913925	total: 10.1s	remaining: 49.1s
17:	learn: 0.4821990	total: 10.4s	remaining: 47.6s
18:	learn: 0.4638384	total: 10.8s	remaining: 46.1s
19:	learn: 0

In [ ]:
test_df_robust = test_df.iloc[:,2:]

test_df_robust = robust_scaler.transform(test_df_robust)

test_df_robust = pd.DataFrame(test_df_robust, columns = test_df.iloc[:,2:].columns, index = test_df.index)

In [ ]:
test_robust = pd.concat([test_df.iloc[:,0:2], test_df_robust], axis=1)

test_robust.head()

,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
TIMESTAMP,,,,,,,,,,,,,,,,,,,,,
2022-09-09 02:01:00,T100306,T_31,0.0,-0.2,0.0,0.0,0.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-09-09 02:09:00,T100304,T_31,0.0,-0.4,0.0,0.0,1.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-09-09 08:42:00,T100304,T_31,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-09-09 10:56:00,T010305,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-09-09 11:04:00,T010306,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
preds = cat2.predict(test_robust)

In [ ]:
submit = pd.read_csv('/content/drive/MyDrive/LG Aimers 2기/데이터/sample_submission.csv')

submit['Y_Class'] = preds

submit.to_csv('/content/drive/MyDrive/LG Aimers 2기/제출/baseline_submission11.csv', index=False)

# Stacking Ensemble

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

In [ ]:
# 개별 ML 모델
knn_clf  = KNeighborsClassifier(n_neighbors=3)
rf_clf = RandomForestClassifier(n_estimators=100, random_state=37)
dt_clf = DecisionTreeClassifier(random_state=37)
ada_clf = AdaBoostClassifier(n_estimators=100)

# 최종 모델
from lightgbm import LGBMClassifier
lgbm_clf = LGBMClassifier()

In [ ]:
train_x_robust.head()

,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
TIMESTAMP,,,,,,,,,,,,,,,,,,,,,
2022-06-13 05:14:00,T050304,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-2.427402,-2.219188,-1.468531,-3.042011,3.639769,NaN,NaN,NaN,NaN,NaN
2022-06-13 05:22:00,T050307,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-2.504955,-1.934317,-0.585859,-2.869835,2.135447,NaN,NaN,NaN,NaN,NaN
2022-06-13 05:30:00,T050304,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-2.453253,-2.845018,-0.698524,-2.705923,3.806916,NaN,NaN,NaN,NaN,NaN
2022-06-13 05:39:00,T050307,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-2.703145,-2.473063,0.055167,-3.525482,1.913545,NaN,NaN,NaN,NaN,NaN
2022-06-13 05:47:00,T050304,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-2.537699,-2.071587,-0.351981,-3.179752,3.409222,NaN,NaN,NaN,NaN,NaN


In [ ]:
train_x_robust = pd.get_dummies(train_x_robust, columns=['LINE','PRODUCT_CODE'])

train_x_robust.head()

,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10,...,X_2875,LINE_T010305,LINE_T010306,LINE_T050304,LINE_T050307,LINE_T100304,LINE_T100306,PRODUCT_CODE_A_31,PRODUCT_CODE_O_31,PRODUCT_CODE_T_31
TIMESTAMP,,,,,,,,,,,,,,,,,,,,,
2022-06-13 05:14:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0,0,1,0,0,0,1,0,0
2022-06-13 05:22:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0,0,0,1,0,0,1,0,0
2022-06-13 05:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0,0,1,0,0,0,1,0,0
2022-06-13 05:39:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0,0,0,1,0,0,1,0,0
2022-06-13 05:47:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0,0,1,0,0,0,1,0,0


In [ ]:
test_robust = pd.get_dummies(test_robust, columns=['LINE','PRODUCT_CODE'])
test_robust.head()

,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10,...,X_2875,LINE_T010305,LINE_T010306,LINE_T050304,LINE_T050307,LINE_T100304,LINE_T100306,PRODUCT_CODE_A_31,PRODUCT_CODE_O_31,PRODUCT_CODE_T_31
TIMESTAMP,,,,,,,,,,,,,,,,,,,,,
2022-09-09 02:01:00,0.0,-0.2,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,NaN,0,0,0,0,0,1,0,0,1
2022-09-09 02:09:00,0.0,-0.4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,NaN,0,0,0,0,1,0,0,0,1
2022-09-09 08:42:00,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,NaN,0,0,0,0,1,0,0,0,1
2022-09-09 10:56:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1,0,0,0,0,0,1,0,0
2022-09-09 11:04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0,1,0,0,0,0,1,0,0


In [ ]:
train_x_robust = train_x_robust.fillna(0)
test_robust = test_robust.fillna(0)

# 개별 모델 학습
knn_clf.fit(train_x_robust, train_y)
rf_clf.fit(train_x_robust, train_y)
dt_clf.fit(train_x_robust, train_y)
ada_clf.fit(train_x_robust, train_y)

AdaBoostClassifier(n_estimators=100)

In [ ]:
knn_pred = knn_clf.predict(test_robust)
rf_pred = rf_clf.predict(test_robust)
dt_pred = dt_clf.predict(test_robust)
ada_pred = ada_clf.predict(test_robust)

In [ ]:
# 개별 예측 결과를 stacking
pred = np.array([knn_pred, rf_pred, dt_pred, ada_pred])
pred = pred.T

print(pred.shape)

(310, 4)


In [ ]:
train_x_robust.head()

,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10,...,X_2875,LINE_T010305,LINE_T010306,LINE_T050304,LINE_T050307,LINE_T100304,LINE_T100306,PRODUCT_CODE_A_31,PRODUCT_CODE_O_31,PRODUCT_CODE_T_31
TIMESTAMP,,,,,,,,,,,,,,,,,,,,,
2022-06-13 05:14:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0,1,0,0,0,1,0,0
2022-06-13 05:22:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0,0,1,0,0,1,0,0
2022-06-13 05:30:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0,1,0,0,0,1,0,0
2022-06-13 05:39:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0,0,1,0,0,1,0,0
2022-06-13 05:47:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0,1,0,0,0,1,0,0


In [ ]:
test_robust.head()

,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10,...,X_2875,LINE_T010305,LINE_T010306,LINE_T050304,LINE_T050307,LINE_T100304,LINE_T100306,PRODUCT_CODE_A_31,PRODUCT_CODE_O_31,PRODUCT_CODE_T_31
TIMESTAMP,,,,,,,,,,,,,,,,,,,,,
2022-09-09 02:01:00,0.0,-0.2,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0,0,0,0,0,1,0,0,1
2022-09-09 02:09:00,0.0,-0.4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0,0,0,1,0,0,0,1
2022-09-09 08:42:00,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0,0,0,1,0,0,0,1
2022-09-09 10:56:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1,0,0,0,0,0,1,0,0
2022-09-09 11:04:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,1,0,0,0,0,1,0,0


In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
xgb_model = XGBClassifier(n_estimators=100)

# 후보 파라미터 선정
params = {'max_depth':[5,7], 'min_child_weight':[1,3], 'colsample_bytree':[0.5,0.75]}

# gridsearchcv 객체 정보 입력(어떤 모델, 파라미터 후보, 교차검증 몇 번)
gridcv = GridSearchCV(xgb_model, param_grid=params, cv=3)

# 파라미터 튜닝 시작
gridcv.fit(train_x_robust, train_y)

print(gridcv.best_params_)f

{'colsample_bytree': 0.5, 'max_depth': 5, 'min_child_weight': 3}


In [ ]:
xgb_model = XGBClassifier(n_estimators=500, colsample_bytree=0.5, max_depth=5, min_child_weight=3)

xgb_model.fit(train_x_robust, train_y)

XGBClassifier(colsample_bytree=0.5, max_depth=5, min_child_weight=3,
              n_estimators=500, objective='multi:softprob')

In [ ]:
xgb_pred = xgb_model.predict(test_robust)

In [ ]:
submit = pd.read_csv('/content/drive/MyDrive/LG Aimers 2기/데이터/sample_submission.csv')

submit['Y_Class'] = xgb_pred

In [ ]:
submit.to_csv('/content/drive/MyDrive/LG Aimers 2기/제출/baseline_submission12.csv', index=False)